In [ ]:
%%capture
!pip install "dask[distributed]"
!pip install planetary-computer

## Imports

In [99]:
import os
import re
import numpy as np
import geopandas as gpd
import xarray as xr
import pystac_client
import stackstac
import rioxarray
from shapely.geometry import mapping

from pystac_client import Client
from stackstac import stack

from rasterio.enums import Resampling
import planetary_computer

## Functions

In [100]:
def get_utm_epsg(lat, lon):
    zone = int((lon + 180) / 6) + 1
    if lat >= 0:
        return 32600 + zone  # Northern Hemisphere
    else:
        return 32700 + zone  # Southern Hemisphere


def extract_month(filename):
    match = re.match(r'(\d{4}_\d{2})/raster_\d{2}_\d+\.tif$', filename)
    if match:
        return match.group(1)
    return None



## Making grids, skip if you already have the grids

In [101]:
# india_boundary = gpd.read_file("./india_boundary.geojson")

In [102]:
# utm_crs = india_boundary.estimate_utm_crs()
# india_boundary = india_boundary.to_crs(utm_crs)

# # 10 meters per pixel → 2240m grid size
# grid_size = 2240

# minx, miny, maxx, maxy = india_boundary.total_bounds

# grid_tiles = []
# for x in np.arange(minx, maxx, grid_size):
#     for y in np.arange(miny, maxy, grid_size):
#         grid_tiles.append(box(x, y, x + grid_size, y + grid_size))

# grid_gdf = gpd.GeoDataFrame({"geometry": grid_tiles}, crs=utm_crs)

# india_grid = gpd.clip(grid_gdf, india_boundary)

# india_grid = india_grid.to_crs(epsg=4326)

# india_grid.to_file("india_grid.geojson", driver="GeoJSON")

# print(f"Created {len(india_grid)} grid tiles.")

### Generating random grids

In [103]:
# # Load data 
# india_grid = gpd.read_file("india_grid.geojson")
# bboxes = gpd.read_file("bbox.geojson")  

# #  Ensure CRS match
# if india_grid.crs != bboxes.crs:
#     bboxes = bboxes.to_crs(india_grid.crs)

# # Select all grids intersecting the bounding boxes (fully or partially) 
# bbox_union = bboxes.unary_union
# grids_in_bbox = india_grid[india_grid.geometry.intersects(bbox_union)]

# # Identify square-like grids (not just 4-sided, but width ≈ height) 
# def is_square_like(geom, tolerance=0.1):
#     if geom.geom_type != "Polygon":
#         return False
#     coords = list(geom.exterior.coords)
#     if len(coords) - 1 != 4:
#         return False
#     minx, miny, maxx, maxy = geom.bounds
#     width = maxx - minx
#     height = maxy - miny
#     if height == 0:
#         return False
#     aspect_ratio = width / height
#     return abs(aspect_ratio - 1) <= tolerance  # e.g. between 0.90 and 1.1

# # Apply the square-like check
# perfect_grids = india_grid[india_grid.geometry.apply(is_square_like)]

# # Remove any square-like grids that intersect with the bounding boxes 
# perfect_grids_outside = perfect_grids[~perfect_grids.geometry.intersects(bbox_union)]

# # Sampling logic 
# TOTAL = 300_000
# n_from_outside = TOTAL - len(grids_in_bbox)

# if n_from_outside < 0:
#     raise ValueError(f"Too many bbox-intersecting grids: {len(grids_in_bbox)} exceeds total {TOTAL}")

# grids_outside_random = perfect_grids_outside.sample(n=n_from_outside, random_state=42)

# # Combine and save
# final_grids = gpd.GeoDataFrame(
#     pd.concat([grids_in_bbox, grids_outside_random], ignore_index=True),
#     crs=india_grid.crs
# )

# final_grids.to_file("india_random_grids.geojson", driver="GeoJSON")


# print(f" Final grid count: {len(final_grids)}")
# print(f" BBox-intersecting grids (all kept): {len(grids_in_bbox)}")
# print(f" Random perfect-square grids from outside: {len(grids_outside_random)}")


### check the grids

In [104]:

# random_grids = gpd.read_file("./irrigation_raw_labels/partial_labels_grids.geojson")
# grid = random_grids  

# ma = leafmap.Map(center=[22.5, 78.9], zoom=4)
# ma.add_basemap('SATELLITE')
# ma.add_geojson(random_grids)
# ma


## Add Unique FID Column to Grid GeoJSON

In [ ]:
def assign_fids_by_geometry(geojson_path, output_path):
    # Load the GeoJSON file
    gdf = gpd.read_file(geojson_path)

    # Create a geometry hash for each row
    def geometry_hash(geom):
        return hashlib.md5(geom.wkb).hexdigest()

    gdf['geom_hash'] = gdf['geometry'].apply(geometry_hash)

    # Map each unique geometry hash to a consistent FID like tile000000, tile000001, ...
    hash_to_fid = {
        h: f"tile{str(i).zfill(6)}"
        for i, h in enumerate(gdf['geom_hash'].unique())
    }

    # Apply the mapping
    gdf['FID'] = gdf['geom_hash'].map(hash_to_fid)

    # Drop the helper column
    gdf = gdf.drop(columns=['geom_hash'])

    # Save to disk
    gdf.to_file(output_path, driver="GeoJSON")
    
    return gdf


In [ ]:
updated_gdf = assign_fids_by_geometry("./rice_practices/mask_ready/partial_labels_grids.geojson", "./rice_practices/mask_ready/partial_labels_grids_FID.geojson")

In [ ]:
grids = gpd.read_file("./rice_practices/mask_ready/partial_labels_grids_FID.geojson")
grids.head(), len(grids)

In [ ]:
print(len(updated_gdf['FID'].unique()))
print(grids[["FID", "filename"]].head(10))

## Downloading S2 pipeline

### Load and Prepare India Grids

In [105]:
# Paths
geojson_path = "./rice_practices/mask_ready/partial_labels_grids_FID.geojson"
output_folder = "./rice_practices/Sentinel2_UTM_TimeSeries_new"
os.makedirs(output_folder, exist_ok=True)

# STAC catalog earth_search API
catalog = Client.open("https://earth-search.aws.element84.com/v1")

# Assets to download
assets = ['green', 'nir', 'red', 'rededge1', 'swir1', 'scl']

In [106]:
# Load grid with FID, filename, and geometry
grids = gpd.read_file(geojson_path)
grids["FID"] = grids["FID"].astype(str)
grids = grids.to_crs("EPSG:4326")
if grids.crs.to_epsg() != 4326:
    raise ValueError("Grid must be in EPSG:4326")
print("Grid CRS:", grids.crs)

Grid CRS: EPSG:4326


### Search, Stack, Composite, and Save

In [ ]:
# Extract unique (FID, yyyymm) pairs
needed_downloads = set()
for _, row in grids.iterrows():
    fid = row["FID"]
    yyyymm = row["filename"].split("/")[0]
    needed_downloads.add((fid, yyyymm))

# Process each tile-month
for fid, yyyymm in sorted(needed_downloads):
    print(f"\nProcessing: {fid} | {yyyymm}")
    year, month = yyyymm.split("_")
    date_range = f"{year}-{month}-01/{year}-{month}-30"
    print(date_range)

    # Match the grid geometry
    match = grids[(grids["FID"] == fid) & (grids["filename"].str.contains(yyyymm))]
    if match.empty:
        print(f"No matching geometry for {fid} in {yyyymm}, skipping.")
        continue

    row = match.iloc[0]
    grid_geom = row.geometry

    # Get UTM EPSG from centroid
    centroid = grid_geom.centroid
    utm_epsg = get_utm_epsg(centroid.y, centroid.x)
    print(f"Using dynamic UTM EPSG for {fid}: {utm_epsg}")

    # STAC search
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=grid_geom,
        datetime=date_range,
        limit=10
    )
    items = search.get_all_items()
    if not items:
        print(f"No images for {fid} in {yyyymm}, skipping.")
        continue

    item = items[0]
    print(f"Sentinel item EPSG: {item.properties.get('proj:epsg') or item.properties.get('proj:code')}")

    # Stack in target UTM zone
    stack = stackstac.stack(
        [item],
        assets=["B02", "B03", "B04", "B08", "scl"],
        epsg=utm_epsg,
        resolution=10,
        chunksize=2048
    )

    # Reproject grid to UTM
    grid_utm = gpd.GeoSeries([grid_geom], crs="EPSG:4326").to_crs(f"EPSG:{utm_epsg}").geometry[0]

    # Check overlap
    grid_bounds = grid_utm.bounds
    stack_bounds = stack.rio.bounds()
    print(f"Grid bounds (EPSG:{utm_epsg}): {grid_bounds}")
    print(f"Stack bounds (EPSG:{utm_epsg}): {stack_bounds}")

    grid_box = box(*grid_bounds)
    stack_box = box(*stack_bounds)

    if not grid_box.intersects(stack_box):
        print(f" No overlap between grid and image for {fid}, skipping.")
        continue

    # Clip to grid
    clipped = stack.rio.clip([grid_utm], f"EPSG:{utm_epsg}", drop=True)

    # Apply SCL quality mask
    scl = clipped.sel(band="scl")
    valid_mask = scl.isin([3, 4, 5, 6, 7])  #Cloud shadow, vegetation an non veg, water, unclassified, 

    
    valid_ratio = valid_mask.sum() / (valid_mask.sizes["x"] * valid_mask.sizes["y"])
    valid_ratio_val = float(valid_ratio.values)

    if valid_ratio_val < 0.2:
        print(f"Low valid pixel ratio ({valid_ratio_val:.2f}) for {fid}, skipping.")
    continue

    # Compute median
    clipped = clipped.where(valid_mask, np.nan)
    median_img = clipped.median(dim="time", keep_attrs=True)

    # Remove SCL band
    if "scl" in median_img.band.values:
        median_img = median_img.sel(band=median_img.band != "scl")

    # Resample to 224x224
    median_img = median_img.rio.reproject(
        dst_crs=f"EPSG:{utm_epsg}",
        shape=(224, 224),
        resampling=1
    )

    # Save output
    output_name = f"{fid}_{yyyymm}_s2.tif"
    output_path = os.path.join(output_folder, output_name)

    median_img.rio.to_raster(output_path)
    print(f"Saved: {output_path} | Shape: {median_img.shape}")



Processing: tile000000 | 2024_07
2024-07-01/2024-07-30
Using dynamic UTM EPSG for tile000000: 32644
Sentinel item EPSG: EPSG:32644
Grid bounds (EPSG:32644): (522449.2626114944, 1848609.2568442666, 524689.2626114952, 1850849.2568442672)
Stack bounds (EPSG:32644): (499975.0, 1789945.0, 610345.0, 1900325.0)
Low valid pixel ratio (0.00) for tile000000, skipping.

Processing: tile000000 | 2024_08
2024-08-01/2024-08-30
Using dynamic UTM EPSG for tile000000: 32644
Sentinel item EPSG: EPSG:32644
Grid bounds (EPSG:32644): (522449.2626114944, 1848609.2568442666, 524689.2626114952, 1850849.2568442672)
Stack bounds (EPSG:32644): (499975.0, 1789945.0, 610345.0, 1900325.0)
Low valid pixel ratio (0.00) for tile000000, skipping.

Processing: tile000000 | 2024_09
2024-09-01/2024-09-30
Using dynamic UTM EPSG for tile000000: 32644
Sentinel item EPSG: EPSG:32644
Grid bounds (EPSG:32644): (522449.2626114944, 1848609.2568442666, 524689.2626114952, 1850849.2568442672)
Stack bounds (EPSG:32644): (499975.0, 1